<a href="https://colab.research.google.com/github/plmsrkr/LLM-Advenures/blob/main/Mistral_7B_Instruct_Demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
 !pip -q install git+https://github.com/huggingface/transformers # need to install from github
!pip install -q datasets loralib sentencepiece
!pip -q install bitsandbytes accelerate xformers einops
!pip -q install langchain

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 6.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 45.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 75.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 32.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.6/519.6 kB 8.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 8.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 16.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 24.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 19.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.6/92.6 MB 19.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 258.1/258.1 kB 31.

In [ ]:
!nvidia-smi

Thu Sep 28 06:56:19 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.105.17   Driver Version: 525.105.17   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA A100-SXM...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   34C    P0    42W / 400W |      0MiB / 40960MiB |      0%      Default |
|                               |                      |             Disabled |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

## Mistral 7B Instruct



In [ ]:
import torch
import transformers
from transformers import AutoTokenizer, AutoModelForCausalLM

In [ ]:
torch.set_default_device('cuda')

In [ ]:


model = AutoModelForCausalLM.from_pretrained("mistralai/Mistral-7B-Instruct-v0.1",
                                             torch_dtype="auto")

tokenizer = AutoTokenizer.from_pretrained("mistralai/Mistral-7B-Instruct-v0.1",
                                          torch_dtype="auto")



Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [ ]:
text = "<s>[INST] What is your favourite condiment? [/INST]Well, I'm quite partial to a good squeeze of fresh lemon juice. It adds just the right amount of zesty flavour to whatever I'm cooking up in the kitchen!</s> [INST] Do you have mayonnaise recipes? [/INST]"

encodeds = tokenizer(text, return_tensors="pt", add_special_tokens=False)

device = 'cuda'
model_inputs = encodeds.to(device)
model.to(device)

generated_ids = model.generate(**model_inputs, max_new_tokens=1000, do_sample=True)
decoded = tokenizer.batch_decode(generated_ids)
print(decoded[0])

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


<s>[INST] What is your favourite condiment? [/INST]Well, I'm quite partial to a good squeeze of fresh lemon juice. It adds just the right amount of zesty flavour to whatever I'm cooking up in the kitchen!</s> [INST] Do you have mayonnaise recipes? [/INST] Of course! Here's a simple recipe I love using:

Ingredients:
- 1 egg
- 1 tablespoon white wine vinegar
- 2 tablespoons olive oil
- 1 teaspoon dijon mustard
- Salt and pepper to taste

Instructions:
1. Place the egg in a small bowl.
2. Add the olive oil, white wine vinegar, dijon mustard, salt, and pepper.
3. Whisk the mixture together with a fork until well combined.
4. Let the mayonnaise sit for a few minutes to allow the flavors to meld.
5. Give it another whisk and serve!

This classic recipe is perfect for sandwiches, salads, and as a dip for vegetables or fish. Enjoy!</s>


### Prompt Format
```
<s>[INST] <user_prompt> [/INST]

<assistant_response> </s>

[INST] <user_prompt>[/INST]
```

In [ ]:
import textwrap

def wrap_text(text, width=90): #preserve_newlines
    # Split the input text into lines based on newline characters
    lines = text.split('\n')

    # Wrap each line individually
    wrapped_lines = [textwrap.fill(line, width=width) for line in lines]

    # Join the wrapped lines back together using newline characters
    wrapped_text = '\n'.join(wrapped_lines)

    return wrapped_text

In [ ]:
def generate(input_text, system_prompt="",max_length=512):
    prompt = f"""<s>[INST]{input_text}[/INST]"""
    inputs = tokenizer(input_text, return_tensors="pt", add_special_tokens=False)
    model_inputs = encodeds.to(device)
    model.to(device)
    outputs = model.generate(**inputs,
                             max_length=max_length,
                             temperature=0.1,
                             do_sample=True)
    text = tokenizer.batch_decode(outputs)[0]
    wrapped_text = wrap_text(text)
    print(wrapped_text)

## CodeGen

In [ ]:
generate('''```python
def print_prime(n):
   """
   Print all primes between 1 and n
   """''', system_prompt="You are a genius python coder")

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


```python
def print_prime(n):
   """
   Print all primes between 1 and n
   """
   if n > 100:
       # Stop if n is greater than 100, since all primes are bigger
       # than 100
       return
   k = 1 # 1 is prime, since all primes are bigger than 1
   while k <= n:
       isprime = True # Assume k is prime
       i = 1
       while i <= k/2:
           if k%i == 0: # If k divides by i
               isprime = False # k is not prime
               break
           if isprime == False:
               break # End of inner while loop
           i = i+1
       if isprime == True: # If k is prime
           # Print k
           print(k)
       k = k + 2 # k should be incremented by 1 or 2

# Test program
print_prime(109)</s>


In [ ]:
generate('''```python
def detect_prime(n):
   """
   detect if a number is a prime number or not. return True or False
   """''')

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


```python
def detect_prime(n):
   """
   detect if a number is a prime number or not. return True or False
   """
   if n <= 1:
      return False
   for k in range(2,int(n**0.5)+1):
      if n % k == 0:
         return False
   return True
num=input ("Enter your number: ")
n=int(num)
if detect_prime(n):
   print(n,"is a prime number.")
else:
   print(n,"is not a prime number.")
```output
```
```</s>


## Instruction Answering

In [ ]:
generate('Write a detailed analogy between mathematics and a lighthouse.',
         max_length=128)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Write a detailed analogy between mathematics and a lighthouse.
Answer: Mathematics is like a lighthouse guiding ships safely to their destinations. Just
as a lighthouse uses a beam of light to signal the location and danger of rocks and
shoals, mathematics uses logic and laws to provide guidance and warn of potential
pitfalls. Both the lighthouse and mathematics are essential tools for those navigating
unknown waters, whether it's the ocean or the unknowns of a problem. The lighthouse
provides a physical presence and a steady light to guide the way, while mathematics
provides


In [ ]:
%%time
generate('Write a detailed analogy between mathematics and a music.',
         max_length=256)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Write a detailed analogy between mathematics and a music.

Mathematics is a language. It's a system of symbols and rules that we use to understand
the world around us. And just like any other language, there are different ways to
approach it, different ways to express it.

One way to think of mathematics is like music. Just as each person has their own way of
listening to music, so too do we have our own way of approaching mathematics. Some people
might prefer to work through problems using logical reasoning, while others might prefer
to work through them using more intuitive or creative methods.

In terms of the rules and symbols that make up mathematics, it's a bit like reading sheet
music. Just as sheet music gives us instructions for playing an instrument, mathematical
symbols give us instructions for solving problems.

And just like music, mathematics can be incredibly beautiful. There are patterns and
formulas that, when used correctly, can produce elegant and profound results. I

In [ ]:
%%time
generate('What is the difference between a Llama, Vicuna and an Alpaca?',
         max_length=512)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


What is the difference between a Llama, Vicuna and an Alpaca?

Llamas, vicuñas, and alpacas are all members of the camelid family, which also includes
camels and guanacos. They are native to the Andes Mountains in South America.

Llamas are the largest of the three, with males weighing up to 400 pounds and standing up
to 4 feet tall at the shoulder. They have long, thick fleece that is typically white or
gray.

Vicuñas are the smallest of the three, with males weighing up to 150 pounds and standing
up to 3 feet tall at the shoulder. They have very fine, soft fleece that is typically
white.

Alpacas are intermediate in size between llamas and vicuñas, with males weighing up to 200
pounds and standing up to 3.5 feet tall at the shoulder. They have medium-length fleece
that is typically white or gray.

In terms of temperament, llamas are generally docile and easy to handle, while vicuñas are
more independent and can be more difficult to work with. Alpacas are known for being
friendly and 

In [ ]:
%%time
generate('Write a short email to Sam Altman giving reasons to open source GPT-4',
         max_length=512)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Write a short email to Sam Altman giving reasons to open source GPT-4.

Dear Sam,

I hope this email finds you in good health and high spirits. I have been following
OpenAI's progress with great interest and believe that GPT-4 is a monumental achievement
that has the potential to revolutionize the field of artificial intelligence.

As a researcher and a developer, I believe that the benefits of open sourcing GPT-4 would
be immense. By making the code and models available to the public, we can accelerate the
pace of innovation and enable researchers and developers across the world to build upon
and extend the capabilities of GPT-4. This would not only result in a more open and
inclusive community, but also empower individuals and organizations to adapt and innovate
more quickly.

Furthermore, open source GPT-4 can help to overcome some of the challenges that come with
proprietary AI models. For example, concerns about model bias and data privacy can be
addressed through transparency and

In [ ]:
%%time
generate('What is the capital of England?',
         max_length=256)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


What is the capital of England?

London is the capital of England.</s>
CPU times: user 550 ms, sys: 1.21 ms, total: 551 ms
Wall time: 549 ms


In [ ]:
%%time
generate('Can Geoffrey Hinton have a conversation with George Washington? Give the rationale before answering.',
         max_length=256)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Can Geoffrey Hinton have a conversation with George Washington? Give the rationale before
answering.

Geoffrey Hinton is a living person who is a computer scientist and cognitive psychologist.
George Washington was a historical figure who lived from 1732 to 1799 and was the first
President of the United States. They lived in different time periods and have no way of
communicating with each other. Therefore, it is not possible for Geoffrey Hinton to have a
conversation with George Washington.</s>
CPU times: user 4.77 s, sys: 8.34 ms, total: 4.78 s
Wall time: 4.77 s


In [ ]:
generate('Write a story about a Koala playing pool and beating all the camelids.',
         max_length=512)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Write a story about a Koala playing pool and beating all the camelids.

Once upon a time in Australia, a small town was overrun with camelids. They ate all the
grass and made a mess everywhere. The locals were desperate for a solution and decided to
hold a competition. The winner would receive a large sum of money and the title of the
ultimate camelid hunter.

Ever since he was a little koala, Koala always had a love for playing pool. He spent most
of his days practicing his shots and perfecting his game. When he heard about the
competition, he knew what he had to do.

The day of the competition arrived and Koala stepped up to the table ready to show off his
skills. The other camelids were not impressed and laughed at him, thinking he was just a
silly little koala.

However, Koala was determined to prove them wrong. He took a deep breath, focused on his
target, and sunk his cue ball into the pocket. The crowd gasped in shock as Koala made
shot after shot, clearing every ball off the ta

## Chat

In [ ]:
generate("""Alice: I don't know why, I'm struggling to maintain focus while studying. Any suggestion? \n\n Bob:""",
         max_length=128)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Alice: I don't know why, I'm struggling to maintain focus while studying. Any suggestion?

 Bob: Well, have you tried breaking your study sessions into smaller chunks? It can help
you stay focused and retain more information.

 Alice: That's a good idea, I'll give it a try.

 Bob: Also, make sure you're taking breaks and doing something enjoyable during those
breaks. It can help you recharge and come back to studying with renewed energy.

 Alice: Yeah, I've been doing that, but I'm still having trouble staying


In [ ]:
generate("""Alice: I don't know why, I'm struggling to maintain focus while studying. Any suggestion? \n\n Bob:""",
         max_length=128)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Alice: I don't know why, I'm struggling to maintain focus while studying. Any suggestion?

 Bob: Well, have you tried breaking your study sessions into smaller chunks? Maybe 25
minutes of intense focus followed by a 5 minute break.

 Alice: That's a good idea, I'll give it a try.

 Bob: Also, make sure you're taking care of your physical health. Eating well, getting
enough sleep, and exercising regularly can all help improve your focus and productivity.

 Alice: Thanks for the advice, I'll make


## GSM8K

In [ ]:
generate('Answer the following question by reasoning step by step. The cafeteria had 23 apples. If they used 20 for lunch, and bought 6 more, how many apple do they have?',
         max_length=256)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Answer the following question by reasoning step by step. The cafeteria had 23 apples. If
they used 20 for lunch, and bought 6 more, how many apple do they have?
Answer: They have 29 apples.

Explanation:

1. The cafeteria started with 23 apples.
2. They used 20 apples for lunch, leaving them with 23 - 20 = 3 apples.
3. Then, they bought 6 more apples, adding them to the remaining 3 apples.
4. So, the total number of apples they have now is 3 (remaining from lunch) + 6 (bought) =
29 apples.</s>


In [ ]:
generate("Weng earns $12 an hour for babysitting. Yesterday, she just did 50 minutes of babysitting. How much did she earn?",
         max_length=128)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Weng earns $12 an hour for babysitting. Yesterday, she just did 50 minutes of babysitting.
How much did she earn?
A: $6</s>


In [ ]:
generate("A deep-sea monster rises from the waters once every hundred years to feast on a ship and sate its hunger. Over three hundred years, it has consumed 847 people. Ships have been built larger over time, so each new ship has twice as many people as the last ship. How many people were on the ship the monster ate in the first hundred years?",
         max_length=256)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


A deep-sea monster rises from the waters once every hundred years to feast on a ship and
sate its hunger. Over three hundred years, it has consumed 847 people. Ships have been
built larger over time, so each new ship has twice as many people as the last ship. How
many people were on the ship the monster ate in the first hundred years?
A: 14</s>
